In [1]:
import os
import glob
from PIL import Image
import cv2
import pandas as pd
import shutil
from tqdm.auto import tqdm
foldername = os.path.basename(os.getcwd())
if foldername == "tools": os.chdir("..")
data_dir = '/custom_dataset/images/train'
data_dir_2 = '/custom_dataset/images/val'
data_dir_3 = '/custom_dataset/labels/train'
data_dir_4 = '/custom_dataset/labels/val'


In [2]:
# Yolv6에 input에 맞게 변경해 주기
img_path = os.path.join(os.getcwd()+data_dir)
for i, image in tqdm(enumerate(glob.glob(img_path+'/*.png'))):
    original_image = cv2.imread(image)
    img_height, img_width, _ =  original_image.shape
    image_name = image.split('\\')[-1].split('.')[0]
    txt_path = os.path.join(os.getcwd()+data_dir_3+'/'+image_name+'.txt')
    tree = open(txt_path)
    bboxes = []
    for line in tree.readlines():
        scores = line.split(' ')
        x, y = ((int(scores[1]) + int(scores[3])) / 2) / img_width, ((int(scores[4]) + int(scores[6])) / 2) / img_height
        w, h = (int(scores[3]) - int(scores[1])) / img_width, (int(scores[6]) - int(scores[4])) / img_height
        bboxes.append(str(scores[0]+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)))
    with open(txt_path, "w") as file:
        for bbox in bboxes:
            file.write(bbox+'\n')

0it [00:00, ?it/s]

In [4]:
# train과 validation 나누기
img_path = os.path.join(os.getcwd()+data_dir)
for i, image in enumerate(glob.glob(img_path+'/*.png')):
    if i % 10 == 0:
        image_name = image.split('\\')[-1].split('.')[0]+'.png'
        re_path = os.path.join(os.getcwd()+data_dir_2+'/'+image_name)
        shutil.move(image, re_path)
        
img_path = os.path.join(os.getcwd()+data_dir_3)
for i, image in enumerate(glob.glob(img_path+'/*.txt')):
    if i % 10 == 0:
        image_name = image.split('\\')[-1].split('.')[0]+'.txt'
        re_path = os.path.join(os.getcwd()+data_dir_4+'/'+image_name)
        shutil.move(image, re_path)

In [5]:
# 이름 잘못 만들어 진거 수정하기
img_path = os.path.join(os.getcwd()+data_dir_2)
for image in glob.glob(img_path+'/*.png'):
    image_name = (image.split('\\')[-1].split('.')[0]+'.png')[3:]
    os.rename(image, os.path.join(img_path, image_name))

In [ ]:
# 한솔 Daicon 대회 Image Rename
number = 0
for i in range(19):
    print(os.getcwd()+data_dir+str(i))
    img_path = os.path.join(os.getcwd()+data_dir+str(i))
    for image in glob.glob(img_path+'/*.png'):
        image_name = image.split('\\')[-1].split('.')[0]
        src = os.path.join(img_path, image)
        dst = str(number).zfill(4) + '.png'
        dst = os.path.join(img_path, dst)
        os.rename(src, dst)
        try: 
            src = os.path.join(img_path, image_name + '.txt')
            dst = str(number).zfill(4) + '.txt'
            dst = os.path.join(img_path, dst)
            os.rename(src, dst)
            number += 1
        except:
            number += 1
            continue

In [41]:
# 한솔 Daicon 대회 Submission
txt_dir = '/runs/inference/exp/test/labels'
txt_path = os.path.join(os.getcwd()+txt_dir)
test_preds = []
data_dir = '/runs/inference/exp/test'
img_path = os.path.join(os.getcwd()+data_dir)
for image in glob.glob(img_path+'/*.png'):
    image_name = image.split('\\')[-1].split('.')[0]
    txt_file = os.path.join(txt_path+'/'+image_name+'.txt')
    best_conf, pred_class = 0, 18
    try:
        labels = open(txt_file).readlines()
        for label in labels:
            scores = label.split()
            if float(scores[-1]) > best_conf: 
                best_conf = float(scores[-1])
                pred_class = int(scores[0])
        test_preds.append(str(pred_class))
    except:
        test_preds.append(str(pred_class))

In [12]:
# 자동차 Detection 대회 Submission
results = pd.read_csv('./custom_dataset/sample_submission.csv')

txt_dir = '/runs/inference/exp/test/labels'
txt_path = os.path.join(os.getcwd()+txt_dir)
data_dir = '/runs/inference/exp/test'
img_path = os.path.join(os.getcwd()+data_dir)
for image in tqdm(glob.glob(img_path+'/*.png')):
    image_name = image.split('\\')[-1].split('.')[0]
    original_image      = cv2.imread(image)
    img_height, img_width, _ =  original_image.shape
    txt_file = os.path.join(txt_path+'/'+image_name+'.txt')
    try:
        labels = open(txt_file).readlines()
        for label in labels:
            scores = label.split()
            class_ind, score = scores[0], scores[-1]
            (x1, y1), (x2, y2) = (float(scores[1]) * img_width, float(scores[2]) * img_height), (float(scores[3]) * img_width, float(scores[4]) * img_height)
            results = results.append({
                                "file_name": image_name+'.png',
                                "class_id": class_ind,
                                "confidence": score,
                                "point1_x": x1, "point1_y": y1,
                                "point2_x": x2, "point2_y": y1,
                                "point3_x": x2, "point3_y": y2,
                                "point4_x": x1, "point4_y": y2
                            }, ignore_index=True)
    except:
        continue
        
results.to_csv('baseline_submit.csv', index=False)
print('Done.')

  0%|          | 0/3400 [00:00<?, ?it/s]

Done.
